In [4]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
from langdetect import detect

In [5]:
#Load data
#group the reviews as low, neutral and high based on rating
#stratify 10000
category = ['bad','neutral','good']

def cat_y(y):
    
    if y<=2.0:
        return category[0]
    
    elif y>=4.0:
        return category[2]
    else:
        return category[1]

def get_reviews(path, n_samples,s=10000):
    dt = {}
    i=0
    with gzip.open(path) as f:
        for l in f:
#             print(l)
            dt[i] = eval(l)
            i += 1
            if (n_samples is not None) and (i > n_samples):
                break
    df = pd.DataFrame.from_dict(dt, orient='index')[['book_id','review_text','rating']]
    
    #remove reviews with less than 20 words
#     df = df[df['review_text'].apply(lambda x: len(x.split())>=20)]
    
    df['sentiment'] = df['rating'].apply(cat_y)
    
    df = df.groupby('sentiment').apply(lambda x: x.sample(n=s))
    
    return df

def load_data(path,cols,n_samples = 1000):
    dt = {}
    i=0
    with gzip.open(path) as f:
        for l in f:
#             print(l)
            dt[i] = eval(l)
            i += 1
            if (n_samples is not None) and (i > n_samples):
                break
                
    df = pd.DataFrame.from_dict(dt, orient='index')[cols]
    
    return df
def get_author_info(x):
#     print(x)
    ll = []
    llnames=[]
    if x:
        for i in x:
#             print(i['author_id'])
            ll.append(i['author_id'])
            llnames.append(d_a[str(i['author_id'])])
        
        return ll, llnames
    else:
        return None, None

In [6]:
path = "data/goodreads_reviews_dedup.json.gz"
#change the third parameter if you want to load more rows into dataframe
review = get_reviews(path, 2000000,250000)
# review = get_reviews(path, 100,10)
pd.set_option('display.max_colwidth', -1)
review.sample(3)

/var/folders/h5/6b6s2cwd2m19kydv6p77mmdh0000gn/T/ipykernel_2228/88482890.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


book_id  \
sentiment                     
good      1035308  1519358    
bad       1323175  8465962    
          991407   26030697   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             review_text  \
sentiment                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
good      1035308  Fun and bright illustrations with a moral at the end. The kids really enjoyed it. \n We'll read it again.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
bad       1323175  I did not like this book, partly because I thought it was self-consciously arty but also because I could not stop judging the author for leaving her children. I don't think that is a fair way to evaluate a book, but I could not get past it. Gloria Steinem would be very disappointed in me (although I don't think a father should voluntarily leave a three-year-old and a five-year-old for six months either). \n I was disappointed because I thought the book would be more about Hiroshima than about the author's maternal and marital woes. In fact, by the end of the book I thought it was a bit self-absorbed to try to juxtapose the deaths of hundreds of thousands of people against one woman's failure to answer her cell phone in a timely manner. (And see, that last sentence isn't entirely fair, either. But this book made me irritable.)   
          991407   I guess that Mike Carey's work under this name is probably just not aimed at me. That's fine, since his output is huge and I can barely keep up anyway, but a shame since I love so much of his other work. \n Fellside is set in a women's prison, with a slight supernatural element. I gave it over 150 pages but I just couldn't bring myself to care about what was happening in this novel. The mystery element

In [7]:
#numer of unique books for which we have reviews
print(len(review.book_id.unique()))

329858


In [9]:
#drop na's from the dataframe.
review['review_text'].dropna(inplace=True)

In [10]:
#Load author information; takes a long time about 20 mins
path = "data/goodreads_books.json.gz"
#change the third parameter if you want to load more rows into dataframe
authors = load_data(path, cols = ['book_id','language_code','authors'],n_samples=500000)
# pd.set_option('display.max_colwidth', -1)
authors.language_code.unique()

array(['', 'eng', 'ger', 'spa', 'en-US', 'ita', 'per', 'en-GB', 'tur',
       'ind', 'mon', 'fre', 'por', 'ara', 'en-CA', 'tha', 'lav', 'jpn',
       'pol', 'swe', 'kor', 'fin', 'msa', 'bul', 'nl', 'gre', 'slo',
       'nor', 'heb', 'hun', 'ben', 'scr', 'zho', 'fil', 'rus', 'lit',
       'rum', 'cze', 'dan', 'slv', 'nno', 'pes', 'hye', 'nob', 'cat',
       'en', 'vie', 'nep', 'mar', 'srp', 'urd', 'guj', 'est', 'sqi',
       'ukr', 'afr', 'mul', 'grc', 'kat', 'mkd', 'hin', 'tam', 'mus',
       '--', 'bos', 'enm', 'gla', 'isl', 'glg', 'mal', 'kur', 'wel',
       'pt-BR', 'crh', 'tel', 'es-MX', 'kan', 'mya', 'fao', 'aze', 'ota',
       'arw', 'pra', 'tgl', 'lat', 'dum', 'eus', 'sin', 'mlt', 'ada',
       'apa', 'udm', 'peo', 'bel', 'iro', 'nld', 'ori', 'smn', 'amh',
       'tut', 'frs', 'arg', 'ang', 'abk', 'epo', 'snd', 'pan', 'egy',
       'dut', 'vls', 'jav', 'tlh', 'din', 'gle', 'alg', 'gsw', 'nah',
       'her', 'aus', 'aka', 'chm', 'ace', 'oci', 'ast', 'kok', 'tib',
       'frm', 'i

In [11]:
# authors.drop(columns=['author_id'], inplace=True)
authors.head()

,book_id,language_code,authors
0,5333265,,"[{'author_id': '604031', 'role': ''}]"
1,1333909,,"[{'author_id': '626222', 'role': ''}]"
2,7327624,eng,"[{'author_id': '10333', 'role': ''}]"
3,6066819,eng,"[{'author_id': '9212', 'role': ''}]"
4,287140,,"[{'author_id': '149918', 'role': ''}]"


In [12]:

# next(iter((authors['authors'][0][0].items())))[1]
# (authors['authors'][0])
# ll=[]
# l = [{'author_id': '381844', 'role': ''}, {'author_id': '238242', 'role': ''}, {'author_id': '4097746', 'role': 'Contributor'}]
# for i in l:
#     print(i['author_id'])
#     ll.append(i['author_id'])

# print(ll)
    

# authors['authors'][0]
# author_info[author_info['author_id'] == '604031']['name'].values[0]


In [13]:
#Load author name information; 
path = "data/goodreads_book_authors.json.gz"
#change the third parameter if you want to load more rows into dataframe
author_info = load_data(path, cols = ['author_id','name'], n_samples=500000)
pd.set_option('display.max_colwidth', -1)
author_info.sample(3)

/var/folders/h5/6b6s2cwd2m19kydv6p77mmdh0000gn/T/ipykernel_2228/3469811614.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,author_id,name
373191,3333238,Zlatan Filipovic
91424,7525130,Amleto De Silva
404724,1190607,Addison Paisley


In [14]:
# create  dict of  author id, name
d_a = dict(zip(author_info.author_id, author_info.name))

In [15]:
# d_a['604031']
# authors_temp = authors[:2]

In [16]:
# authors_temp.head()

In [17]:
authors['author_id'], authors['author_names'] = zip(*authors['authors'].apply(lambda x: get_author_info(x)))
# authors_temp['author_id'], authors_temp['author_names'] = authors_temp['authors'].apply(lambda x: get_author_info(x))

In [18]:
authors.head()

,book_id,language_code,authors,author_id,author_names
0,5333265,,"[{'author_id': '604031', 'role': ''}]",[604031],[Ronald J. Fields]
1,1333909,,"[{'author_id': '626222', 'role': ''}]",[626222],[Anita Diamant]
2,7327624,eng,"[{'author_id': '10333', 'role': ''}]",[10333],[Barbara Hambly]
3,6066819,eng,"[{'author_id': '9212', 'role': ''}]",[9212],[Jennifer Weiner]
4,287140,,"[{'author_id': '149918', 'role': ''}]",[149918],[Nigel Pennick]


In [19]:
#merge author information with reviews 
review = review.merge(authors, how='left', on='book_id')

In [20]:
#Load genre information; 
path = "data/goodreads_book_genres_initial.json.gz"
#change the third parameter if you want to load more rows into dataframe
genres = load_data(path, cols = ['book_id','genres'], n_samples=500000)
pd.set_option('display.max_colwidth', -1)
genres.sample(3)

/var/folders/h5/6b6s2cwd2m19kydv6p77mmdh0000gn/T/ipykernel_2228/3189724887.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,book_id,genres
137414,13481421,"{'fantasy, paranormal': 1}"
98207,28540592,{}
359048,2949077,"{'non-fiction': 6, 'fantasy, paranormal': 1, 'history, historical fiction, biography': 1}"


In [21]:
genres.shape

(500001, 2)

In [22]:
genres['genres'] = genres['genres'].apply(lambda x: list(x.keys()))

In [23]:
genres.head()

,book_id,genres
0,5333265,"[history, historical fiction, biography]"
1,1333909,"[fiction, history, historical fiction, biography]"
2,7327624,"[fantasy, paranormal, fiction, mystery, thriller, crime, poetry]"
3,6066819,"[fiction, romance, mystery, thriller, crime]"
4,287140,[non-fiction]


In [24]:
#merge genre information with reviews 
review = review.merge(genres, how='left', on='book_id')

In [25]:
review.head()

,book_id,review_text,rating,sentiment,language_code,authors,author_id,author_names,genres
0,13526165,"So many people loved this one, but it just didn't work for me. Too over-the-top and bizarre for me. I didn't find any of it funny, just... strange. Stopped reading a little over halfway through, too many other books calling to me!",0,bad,NaN,NaN,NaN,NaN,NaN
1,25785553,"Meh, whatever. some of the explanations of dog behavior are interesting but I got sick of hearing about the authors dog.",2,bad,NaN,NaN,NaN,NaN,NaN
2,12283261,http://librarianosnark.blogspot.com/2...,2,bad,NaN,NaN,NaN,NaN,NaN
3,6186188,"Jackie says: \n This new novel by the author of Time Is A River takes us to the coast of South Carolina and into the world of the shrimping industry, hit hard by the current economy and imports of foreign shrimp to American markets. It's the story of Bud and Carolina, whose decades of marriage is on shaky ground. It's also the story of a down on its luck town who, when emergency strikes, pulls together and takes care of its own while forgetting all the rivalries and resentments and worries that generally plague them. The story moves back and forth between the present and scenes from Bud and Carolina's past, giving depth and background to the characters to this well told story. \n ***Unchain Yourself! Please look for this great book at your local independent book store. There is a store finder at indiebound.org.***",0,bad,NaN,NaN,NaN,NaN,NaN
4,18143977,"I had a problem with the structure of this book and the author's writing style. \n He tells the story in a non-linear fashion, starting in 1944, then back to 1934, back to 1944 and back to 1940. The chapters vary in length from two to five or six pages. That's a minor issue for me. \n Mr. Doerr uses mostly short sentences and many sentence fragments or groups of words that are not a sentence. While reading it I got the feeling that it's coming at me staccato, boom-boom-boom. This became rather tiresome after reading twenty or thirty pages. But I soldiered on and, after 130 or so pages and not feeling compelled to continue, I decided to give up the ghost. \n In fairness, since I haven't finished it, I'm not going to give any stars.",0,bad,NaN,NaN,NaN,NaN,NaN


In [26]:
#Checking to see how many null values are in the dataframe.
review.isnull().sum()

book_id          0     
review_text      0     
rating           0     
sentiment        0     
language_code    587426
authors          587426
author_id        587429
author_names     587429
genres           587426
dtype: int64

In [27]:
#Save file 
review.to_csv('data/reviews_authors_genres.csv')